In [123]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [124]:
def get_web_content(url):
    payload = {}
    headers = {
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
    }
    response = requests.request("GET", url, headers=headers, data = payload)
    return response.text.encode('utf8')

def get_js_head(links):
    data = []
    for link in links:
        data_jobid = link.get("data-job-id")
        data_id = link.get("id")
        data_href = link.get("href")
        data_text = link.text.replace("\n","")
        if data_jobid is not None:
            row = [data_jobid, data_text,data_href]
            data.append(row)
    return data


In [125]:
# get detail vacancy
def get_detail_vacancy(jobid):
    data = []
    url = "https://www.jobstreet.co.id/id/job/"+jobid
    resp = get_web_content(url)
    soup = BeautifulSoup(resp, "lxml")
    divs = soup.find_all("div")
    company_overview_all = ""
    job_description = ""
    company_name = ""
    position = ""
    for div in divs:
        panel = div.get("id") 
        if panel == "company_overview_all":
            company_overview_all = div.text.strip()
        if panel == "job_description":
            job_description = div.text.strip()
        if panel == "company_name":
            company_name = div.text.strip()

    h1s = soup.find_all("h1")
    for h1 in h1s:
        position_title = h1.get("id")
        if position_title == "position_title":
            position = h1.text.strip()

    row = [jobid, position, company_name, company_overview_all, job_description]
    return row

In [129]:
url = "https://www.jobstreet.co.id/id/job-search/job-vacancy.php"
resp = get_web_content(url)

soup = BeautifulSoup(resp, "lxml")
links = soup.find_all("a")
data = get_js_head(links)
final = []
for d in data:
    detail = get_detail_vacancy(d[0])
    final.append(detail)

# print(final)
df = pd.DataFrame(final,columns=['jobid', 'position', 'company_name', 'company_overview_all', 'job_description'])
conn = sqlite3.connect('scrap.js.db')
df.to_sql("JS_DETAIL", conn, if_exists='replace', index = False)
